In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qfqrtd0a
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qfqrtd0a
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=b0471a78210e578dbc9541dca6d5363259f7b2837ac026c59e0bfde379e1500e
  Stored in directory: /tmp/pip-ephem-wheel-cache-jn7r_46g/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext Cython

In [4]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
     

34 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2023-05-26 21:16:56--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2023-05-26 21:16:56--  https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61

In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [8]:
%%cu
#include <stdio.h>

#define N 4 // Size of the matrices

// CUDA kernel to perform matrix multiplication
__global__
void matrixMul(int *a, int *b, int *c, int n)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < n && col < n)
    {
        int sum = 0;
        for (int i = 0; i < n; ++i)
        {
            sum += a[row * n + i] * b[i * n + col];
        }
        c[row * n + col] = sum;
    }
}

// Function to display a matrix
void displayMatrix(int *matrix, int n)
{
    for (int i = 0; i < n; ++i)
    {
        for (int j = 0; j < n; ++j)
        {
            printf("%d ", matrix[i * n + j]);
        }
        printf("\n");
    }
}

int main()
{
    int *a, *b, *c; // Host matrices
    int *d_a, *d_b, *d_c; // Device matrices

    int size = N * N * sizeof(int);

    // Allocate memory for host matrices
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize host matrices
    for (int i = 0; i < N; ++i)
    {
        for (int j = 0; j < N; ++j)
        {
            a[i * N + j] = i + j;
            b[i * N + j] = i - j;
        }
    }

    // Allocate memory for device matrices
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy host matrices to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel on GPU
    dim3 threadsPerBlock(N, N);
    dim3 blocksPerGrid(1, 1);
    if (N * N > 1024)
    {
        threadsPerBlock.x = 32;
        threadsPerBlock.y = 32;
        blocksPerGrid.x = ceil((float)N / threadsPerBlock.x);
        blocksPerGrid.y = ceil((float)N / threadsPerBlock.y);
    }
    matrixMul<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, N);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print the matrices and the product
    printf("Matrix A:\n");
    displayMatrix(a, N);
    printf("\nMatrix B:\n");
    displayMatrix(b, N);
    printf("\nProduct Matrix:\n");
    displayMatrix(c, N);

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(a);
    free(b);
    free(c);

    return 0;
}


Matrix A:
0 1 2 3 
1 2 3 4 
2 3 4 5 
3 4 5 6 

Matrix B:
0 -1 -2 -3 
1 0 -1 -2 
2 1 0 -1 
3 2 1 0 

Product Matrix:
0 0 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 



In [9]:
%%cu
#include <stdio.h>

#define N 10 // Matrix size

// CUDA kernel for matrix multiplication
__global__
void matrixMultiply(int *a, int *b, int *c)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < N && col < N)
    {
        int sum = 0;
        
        for (int k = 0; k < N; ++k)
        {
            sum += a[row * N + k] * b[k * N + col];
        }
        
        c[row * N + col] = sum;
    }
}

// Function to display a matrix
void displayMatrix(int *matrix)
{
    for (int i = 0; i < N; ++i)
    {
        for (int j = 0; j < N; ++j)
        {
            printf("%d\t", matrix[i * N + j]);
        }
        printf("\n");
    }
    printf("\n");
}

int main()
{
    int *a, *b, *c; // Host matrices
    int *d_a, *d_b, *d_c; // Device matrices

    int size = N * N * sizeof(int);

    // Allocate memory for host matrices
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize host matrices
    for (int i = 0; i < N; ++i)
    {
        for (int j = 0; j < N; ++j)
        {
            a[i * N + j] = i + j;
            b[i * N + j] = i - j;
        }
    }

    // Allocate memory for device matrices
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy host matrices to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);

    // Launch kernel on GPU
    matrixMultiply<<<gridDim, blockDim>>>(d_a, d_b, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Display input matrices and product
    printf("Matrix A:\n");
    displayMatrix(a);

    printf("Matrix B:\n");
    displayMatrix(b);

    printf("Product C = A * B:\n");
    displayMatrix(c);

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(a);
    free(b);
    free(c);

    return 0;
}


Matrix A:
0	1	2	3	4	5	6	7	8	9	
1	2	3	4	5	6	7	8	9	10	
2	3	4	5	6	7	8	9	10	11	
3	4	5	6	7	8	9	10	11	12	
4	5	6	7	8	9	10	11	12	13	
5	6	7	8	9	10	11	12	13	14	
6	7	8	9	10	11	12	13	14	15	
7	8	9	10	11	12	13	14	15	16	
8	9	10	11	12	13	14	15	16	17	
9	10	11	12	13	14	15	16	17	18	

Matrix B:
0	-1	-2	-3	-4	-5	-6	-7	-8	-9	
1	0	-1	-2	-3	-4	-5	-6	-7	-8	
2	1	0	-1	-2	-3	-4	-5	-6	-7	
3	2	1	0	-1	-2	-3	-4	-5	-6	
4	3	2	1	0	-1	-2	-3	-4	-5	
5	4	3	2	1	0	-1	-2	-3	-4	
6	5	4	3	2	1	0	-1	-2	-3	
7	6	5	4	3	2	1	0	-1	-2	
8	7	6	5	4	3	2	1	0	-1	
9	8	7	6	5	4	3	2	1	0	

Product C = A * B:
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	
0	0	0	0	0	0	0	0	0	0	


